# **Character-RNN**

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

from torch.nn.utils.rnn import pad_sequence

import string

## Getting Data

In [2]:
df = pd.read_csv('names-and-origins.csv')

In [3]:
df.head(10)

,origin,name,normalized_name
0,English,Abbas,abbas
1,English,Abbey,abbey
2,English,Abbott,abbott
3,English,Abdi,abdi
4,English,Abel,abel
5,English,Abraham,abraham
6,English,Abrahams,abrahams
7,English,Abrams,abrams
8,English,Ackary,ackary
9,English,Ackroyd,ackroyd


In [4]:
len(df)

20074

In [5]:
df['origin'] = df['origin'].astype('category')

In [6]:
df['origin_code'] = df['origin'].cat.codes

In [15]:
df['origin'].cat.categories

Index(['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish',
       'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese'],
      dtype='object')

In [16]:
origins = {i:origin for i, origin in enumerate(df['origin'].cat.categories)}

In [17]:
origins

{0: 'Arabic',
 1: 'Chinese',
 2: 'Czech',
 3: 'Dutch',
 4: 'English',
 5: 'French',
 6: 'German',
 7: 'Greek',
 8: 'Irish',
 9: 'Italian',
 10: 'Japanese',
 11: 'Korean',
 12: 'Polish',
 13: 'Portuguese',
 14: 'Russian',
 15: 'Scottish',
 16: 'Spanish',
 17: 'Vietnamese'}

In [9]:
df.head(10)

,origin,name,normalized_name,origin_code
0,English,Abbas,abbas,4
1,English,Abbey,abbey,4
2,English,Abbott,abbott,4
3,English,Abdi,abdi,4
4,English,Abel,abel,4
5,English,Abraham,abraham,4
6,English,Abrahams,abrahams,4
7,English,Abrams,abrams,4
8,English,Ackary,ackary,4
9,English,Ackroyd,ackroyd,4


## Tokenizer

In [23]:
class Tokenizer:
    
    def __init__(self, num_tokens=26):
        self.num_tokens = num_tokens
        self.tokens = list(string.ascii_lowercase)
        self.token_to_index = {ch:i for i,ch in enumerate(self.tokens)}
        self.index_to_token = {i:ch for i,ch in enumerate(self.tokens)}
        
    def tokenize(self, x):
        one_hot = torch.zeros(self.num_tokens, dtype=torch.long)
        if x not in self.tokens:
            raise Exception("unknown token")
        else:
            idx = self.token_to_index[x]
        one_hot[idx] = 1
        return one_hot
    
    def get_char(self,x):
        idx = torch.argmax(x).item()
        return self.index_to_token[idx]
    
    def tokenize_name(self, name):
        vector = torch.zeros(size=(len(name), self.num_tokens))
        for i,ch in enumerate(name):
            vector[i] = self.tokenize(ch)
            
        return vector
    
    def get_name(self,vector):
        name = ''
        for i in range(vector.size(0)):
            if torch.sum(vector[i]).item() == 1:
                name += self.get_char(vector[i])
            
        return name

In [24]:
tk = Tokenizer()
tk.tokenize_name('shreyas').shape

torch.Size([7, 26])

## Padding

In [14]:
padded = pad_sequence([tk.tokenize_name('shreyas'), tk.tokenize_name('daniel')], batch_first=True)

padded.shape

torch.Size([2, 7, 26])

# Custom Dataset

In [25]:
class NamesDataset:
    
    def __init__(self, df, is_test = False, shuffle=True):
        
        self.is_test = is_test
        self.df = df.sample(frac=1)
        
        self.df['origin'] = self.df['origin'].astype('category')
        self.df['origin_code'] = self.df['origin'].cat.codes
        
        self.names = list(self.df['normalized_name'])
        self.origins = list(self.df['origin'])
        self.categories = {i:origin for i, origin in enumerate(self.df['origin'].cat.categories)}
        self.labels = list(self.df['origin_code'])
        
        self.longest = max(list(map(len, self.names)))
        
        self.tk = Tokenizer()
        
    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, idx):
        
        name = self.tk.tokenize_name(self.names[idx])
        pad_zeros = torch.zeros(size=(self.longest-name.size(0), self.tk.num_tokens))
        
        padded = torch.concat([name,pad_zeros])
        
        if self.is_test is False:
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return (padded, label)
        
        return padded

In [26]:
dataset = NamesDataset(df)
dataset[0][0]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [35]:
tk.get_name(torch.tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]]))

'cann'